# RAG Pipeline

### Pine cone

Pine cone will be the the vector database implemented for storing all the embeddings created for the cord19 documents. 

All the embeddings are stored within an index in Pinecone. For this tutorial we will store the break up documents into two name spaces title and abstract. 

In [1]:
from pinecone import Pinecone, ServerlessSpec
from Keys import api_key
import pinecone
pine = Pinecone(api_key=api_key)

C:\Users\aaron\Projects\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
index = pine.Index("documentvectorsdb")

## Cord 19 Dataset and Biobert embeddings

In [3]:
from datasets import load_dataset

data = load_dataset("cord19", "metadata")

In [4]:
df = data["train"].to_pandas()
df = df[["title", "abstract"]]

In [5]:
from PreTrainedModel import PreTrainedModel
model_name="dmis-lab/biobert-base-cased-v1.1"
column_name="title"
limit=20
model = PreTrainedModel(model_name, df, column_name, limit)

C:\Users\aaron\Projects\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
embeddings = model.process()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


## Upserting the Data to the Database

In [8]:
index.upsert(embeddings, namespace="cord-titles")

{'upserted_count': 21}